# 如何保存和加载LangChain对象
LangChain 类实现了标准的序列化方法。使用这些方法对 LangChain 对象进行序列化具有以下优势：
- 密钥（如API密钥）与其他参数分开存储，并可在反序列化时重新加载到对象中；- 反序列化过程在不同包版本间保持兼容，因此使用某一版本LangChain序列化的对象能够被另一版本正确反序列化。
要使用此系统保存和加载LangChain对象，请使用`langchain-core`的[load模块](https://python.langchain.com/api_reference/core/load.html)中的`dumpd`、`dumps`、`load`和`loads`函数。这些函数支持JSON及可JSON序列化的对象。
所有继承自[Serializable](https://python.langchain.com/api_reference/core/load/langchain_core.load.serializable.Serializable.html)的LangChain对象都支持JSON序列化。示例包括[消息](https://python.langchain.com/api_reference//python/core_api_reference.html#module-langchain_core.messages)、[文档对象](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html)（例如从[检索器](/docs/concepts/retrievers)返回的结果），以及大多数[可运行对象](/docs/concepts/lcel)，例如聊天模型、检索器和使用LangChain表达式语言实现的[链](/docs/how_to/sequence)。
下面我们通过一个简单的[LLM链](/docs/tutorials/llm_chain)示例进行讲解。
:::注意
使用 `load` 和 `loads` 进行反序列化可以实例化任何可序列化的 LangChain 对象。请仅在受信任的输入中使用此功能！
反序列化是一项测试版功能，可能会发生变化。好的,我会严格按照要求进行翻译,确保markdown格式一致。以下是一个示例翻译:

# 欢迎使用翻译助手

这是一个示例markdown文档的翻译。

## 二级标题示例

这里是一些**加粗**和*斜体*的文本示例。

### 三级标题示例

- 列表项1
- 列表项2
- 列表项3

> 这是一个引用块的示例

`这是行内代码的示例`

```python
# 这是代码块的示例
def hello():
    print("Hello World!")
```

[这是一个链接示例](https://example.com)

![图片描述](image.jpg)

| 表头1 | 表头2 |
|-------|-------|
| 内容1 | 内容2 |

In [12]:
from langchain_core.load import dumpd, dumps, load, loads
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Translate the following into {language}:"),
        ("user", "{text}"),
    ],
)

llm = ChatOpenAI(model="gpt-4o-mini", api_key="llm-api-key")

chain = prompt | llm

## 保存对象
### 转为 JSON

In [2]:
string_representation = dumps(chain, pretty=True)
print(string_representation[:500])

{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "schema",
    "runnable",
    "RunnableSequence"
  ],
  "kwargs": {
    "first": {
      "lc": 1,
      "type": "constructor",
      "id": [
        "langchain",
        "prompts",
        "chat",
        "ChatPromptTemplate"
      ],
      "kwargs": {
        "input_variables": [
          "language",
          "text"
        ],
        "messages": [
          {
            "lc": 1,
            "type": "constructor",
         


### 转换为可JSON序列化的Python字典

In [3]:
dict_representation = dumpd(chain)

print(type(dict_representation))

<class 'dict'>


### 到磁盘

In [4]:
import json

with open("/tmp/chain.json", "w") as fp:
    json.dump(string_representation, fp)

请注意，API密钥不会包含在序列化表示中。被视为机密的参数由LangChain对象的`.lc_secrets`属性指定：

In [5]:
chain.last.lc_secrets

{'openai_api_key': 'OPENAI_API_KEY'}

## 加载对象
在 `load` 和 `loads` 中指定 `secrets_map` 会将对应的密钥加载到反序列化后的 LangChain 对象上。
### 从字符串

In [7]:
chain = loads(string_representation, secrets_map={"OPENAI_API_KEY": "llm-api-key"})

### 来自字典

In [9]:
chain = load(dict_representation, secrets_map={"OPENAI_API_KEY": "llm-api-key"})

### 从磁盘

In [10]:
with open("/tmp/chain.json", "r") as fp:
    chain = loads(json.load(fp), secrets_map={"OPENAI_API_KEY": "llm-api-key"})

请注意，我们恢复了指南开头指定的API密钥：

In [11]:
chain.last.openai_api_key.get_secret_value()

'llm-api-key'